# Primitives examples

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [2]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: 0.6071428571428571
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [1]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [IQP(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: 0.1588235294117647
>>> Standard errors for PUB 0: 0.18372570722417006
>>> Expectation values for PUB 1: -0.2426470588235294
>>> Standard errors for PUB 1: 0.24287181286582873
>>> Expectation values for PUB 2: 0.01220703125
>>> Standard errors for PUB 2: 0.015620946358330926


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [5]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.07062932e+00  1.01475593e+00  8.60450256e-01  6.10803180e-01
   3.38093469e-01  6.89501449e-03 -3.48554870e-01 -6.15796122e-01
  -8.53079724e-01 -1.00477004e+00 -1.05826584e+00 -1.00619660e+00
  -8.46184709e-01 -6.17698195e-01 -3.19786017e-01  7.13277361e-04
   3.17883944e-01  6.12229735e-01  8.73289249e-01  1.00976298e+00
   1.06967828e+00]
 [-2.37759120e-03  3.27156550e-01  6.05572479e-01  8.58310424e-01
   1.00786091e+00  1.06397206e+00  1.00857419e+00  8.48324541e-01
   6.15796122e-01  3.00527528e-01 -4.27966417e-03 -3.38331228e-01
  -6.31963742e-01 -8.48324541e-01 -1.01118954e+00 -1.06492310e+00
  -1.01166506e+00 -8.38100899e-01 -6.20313545e-01 -3.13366521e-01
   1.16501969e-02]
 [ 3.09086856e-03 -3.46890557e-01 -6.16271640e-01 -8.30254848e-01
  -9.83371722e-01 -1.03924511e+00 -9.95021918e-01 -8.55457315e-01
  -6.17698195e-01 -3.36191396e-01  4.27966417e-03  3.38093469e-01
   6.16984917e-01  8.49751096e-01  9.97637269e-01  1.05018203e+00
   9.89791218e

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

In [4]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: 0.01123046875
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: -0.34210526315789475
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['1010001101100000111001101101101001000000001011111000010011000100011001100000000000000110010011000000100010110010001010010000100', '0111010100100001100010011100100001101001001100100001000111000100000000000110110100011110010011010001000111011100001000110100100', '1110111010010010000000100100001100010110000010011000100000000101001000001000000001010010011011001001100000010011001000010100101', '1000011111010011100111001101001000000110001001100000100010000000011000000100001000001110001010000001001111011001001100000100000', '1011001110111111010101010000010100100000001101010100010001000010000001000010001100000110011000000100100111111000001000010000000', '1010110010100110101010001101110001001001001101110001100000000110010010011000011010010100001010000000100110101110001000000000001', '0101011010001111000000110101000100000100001011100001010111001100000100000110000000110000010011000001000101110001000001100100010', '11100001001100010000001111001000100110110010111110010100011

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [3]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [IQP(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['0000110100100000111101101000100111000101000010010110100010110100000001100000011010100000001101101000101110100101100000101010000', '1100011010111110001110110001001100101011011101100001100000110100010011000010011000001001000000101000001010001101000000100000000', '0111001110000001000101110111001101100100101110010001000010000010100000010010110010110000100000001001100001000101010100001100000', '0111110001100101100010110110000000011010001011000101000010111000000001010000010000010000101000111000101000000000110000001100000', '1011110011100000111110000000011110111111100101000110001000010010101001000100010011011010000000111000010111000101110000101000000', '1011011010100110000000100000100000101011011010101100100010100001100011101100110000110000010000111000010101001100000000001100100', '1101001000010000011100100000000111111110010000010000100101111000101000010010010010001001000000011000100111000000010000000110000', '0110100001100111000110111100001101110010000110000

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [2]:
import numpy as np
from qiskit.circuit.library import RealAmplitudes
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = RealAmplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['0110111100010000101100010101001000100000100001000110000011100011111011000001101001111010100111001010010000001011110100100000111', '0110011101010001011001001000001000000000100000101110100000001011101001100010101000010011110100011000010000000000100010101100110', '1100000100010011100010000101101100100100101100011100100001101010111010000010001010011100011001000011011101011100000101100001101', '0100001100010001101000000000000111101100111001010010100001001000100000000000000011010001000100010110111111001011100000000011100', '1110110010010001100000000001000010100000101001010110011111101100101000100010010010001111111000011111000110011111100100000101110', '0100100101010000010100010010001101111100100001011100010101001010110001100100111010001000011010111100101111000001100000000010101', '0100011100010101001001010100011100000110100100010100101101111100111011110000101001000000000111100100110011001010100010110001000', '11011111100101011010000001010

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

In [1]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['1111001011111011001010101010000110000110101010110111000100011100000111000000010010000101000000101001010011110001100111100101001', '0011011111100010111011000000101000000000011010010111000001001100101111000000000001001101000100001111010010000001100000101001100', '0010101011011100000010000000000110010000100011100101001001001010110001110000110000110001110011101100010001100100110010001101010', '0110000011000000100011101001011001000001000011000110010101000010110110110100100111000011100100101101110010101001111000101100100', '1001011110110000100111000100000101000010100011110110000100000111110111000100100001000011000101100111010011010000110000000100000', '1000111011011110110110101011000100011011110010011101000100000001000011100100110010001000011100100111010011100100111010100001110', '1111110010110000111001001001001010111000100000010101000000000110100101000000111110100111100000001101110001010001110011111001010', '1010101111000111110110010110001101

In [13]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['1000110010100001110100000101101100000001010010000000000101101101100001000100001001000010111010101010000001010010000010000011010', '0011001000101001101111011101000101001000010010000001000000101000100001001100001000110000110010001001110010110111000010001110111', '0100011100000101110000001001001010000100111000010010100010010101000010000001000001001010100000001001110000010100000000011000001', '0001101001001001110010100111001110011000011010001001000101100100100001000000000001000010100010001000010000100100101010000000101', '1111001110011011010001010010001010000001001010010011010111001011100001011000000000110010100010101001010110001000001000000010101', '0011100011000010010001001000001110111101010100000001100000100111000000010000000000000000010010101101010010011000001110000000011', '0011010001111011110010100110011011010100011000010011011001000100000001000000001010000100100010000010110001000110001000110110001', '0101100010000010011010001000010000

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
    - Read [Migrate to V2 primitives](/migration-guides/v2-primitives).
    - Understand the [Job limits](/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>